In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('kk (1).csv')
df2 = df.dropna()
X = df2[['light sleep (sec)', 'deep (sec)', 'start time from 8pm','end time from 8pm','step count']]
y = df2['Snooze']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22)

print(df.head())

   light sleep (sec)  deep (sec)  start time from 8pm  end time from 8pm  \
0              10140        3300                25260              38700   
1              11760        7620                18900              38280   
2               9540       12420                23220              18900   
3              11880        6300                24540              42720   
4              16920        1500                21180              40680   

   step count  Snooze  
0        9841       5  
1       10123       4  
2        5412       9  
3       11985       4  
4       12111       4  


In [4]:
df.shape

(46, 6)

In [5]:
model = Sequential(
    [ 
        Dense(100, activation = 'relu'),
        Dense(150, activation = 'relu'),
        Dense(50, activation = 'relu'),
        Dense(10, activation = 'softmax')    # < softmax activation here
    ]
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)


In [6]:

model.fit(
    X_train,y_train,
    epochs=1000
)

Epoch 1/1000
2/2 [==============================] - 0s 6ms/step - loss: 3526.6919
Epoch 2/1000
2/2 [==============================] - 0s 3ms/step - loss: 1683.5690
Epoch 3/1000
2/2 [==============================] - 0s 3ms/step - loss: 1476.2179
Epoch 4/1000
2/2 [==============================] - 0s 2ms/step - loss: 1106.3573
Epoch 5/1000
2/2 [==============================] - 0s 4ms/step - loss: 739.0048
Epoch 6/1000
2/2 [==============================] - 0s 3ms/step - loss: 653.3890
Epoch 7/1000
2/2 [==============================] - 0s 2ms/step - loss: 568.9111
Epoch 8/1000
2/2 [==============================] - 0s 2ms/step - loss: 423.5088
Epoch 9/1000
2/2 [==============================] - 0s 2ms/step - loss: 349.1712
Epoch 10/1000
2/2 [==============================] - 0s 3ms/step - loss: 377.5947
Epoch 11/1000
2/2 [==============================] - 0s 1ms/step - loss: 322.6431
Epoch 12/1000
2/2 [==============================] - 0s 2ms/step - loss: 207.3448
Epoch 13/1000
2/2 [==

In [7]:
y_pred=(model.predict(X_test))

1/1 [==============================] - 0s 69ms/step


In [8]:
y_pred

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.00

In [9]:
y_pred_val = np.where(y_pred >= 0.5)

In [10]:
len(y_pred_val[1])

12

In [11]:
y_pred_val[1]

array([4, 3, 3, 4, 4, 4, 4, 4, 7, 8, 3, 3], dtype=int64)

In [12]:
y_test

43    4
11    3
22    3
3     4
33    3
1     4
26    4
27    4
12    7
13    8
45    3
32    3
Name: Snooze, dtype: int64

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_val[1])

0.9166666666666666

In [14]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, y_pred_val[1]))

[[4 1 0 0]
 [0 5 0 0]
 [0 0 1 0]
 [0 0 0 1]]


In [17]:
model.save("model_v3.keras")

v_model = keras.models.load_model("model_v3.keras")

converter = tf.lite.TFLiteConverter.from_keras_model(v_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\navan\AppData\Local\Temp\tmptdv198ai\assets


INFO:tensorflow:Assets written to: C:\Users\navan\AppData\Local\Temp\tmptdv198ai\assets


In [18]:
# # Save the TFLite model to a file
with open('model_v3.tflite', 'wb') as f:
    f.write(tflite_model)